# COGS 108 - Data Checkpoint

# Names

- Jennifer Jiang
- Hannah Ordonez
- Daniela Garcia 
- Ghada Barhoush
- Stephanie Li
- Cynthia Delira


<a id='research_question'></a>
# Research Question

> *How might the experiences of women and men differ in tech in terms of mental health issues/comfortability of discussing mental health issues?*

# Dataset(s)

- Dataset Name: Mental Health in Tech Survey
- Link to the dataset: [Mental Health in Tech](https://www.kaggle.com/datasets/osmi/mental-health-in-tech-survey)
- Number of observations: 1259 Rows (observations

#### Mental Health in Tech
- This dataset includes the results of a 2014 survey that: 
> "measures attitudes towards mental health and frequency of mental health disorders in the tech workplace."

- Features of the dataset include:
    - Age of Individual
    - Gender of Individual
    - Individual's Country of Residence
    - Whether the Individual has sought treatment for mental-health related issues
    - Number of employees in the Individual's organization
    - Etc.


# Setup

In [1]:
import pandas as pd
import numpy as np
import os
import re
import sklearn

In [2]:
tech_data_fp = os.path.join('data', 'survey.csv')
tech_data = pd.read_csv(tech_data_fp)

In [3]:
tech_data

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,...,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,...,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,...,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN
4,2014-08-27 11:30:22,31,Male,United States,TX,NaN,No,No,Never,100-500,...,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,2015-09-12 11:17:21,26,male,United Kingdom,NaN,No,No,Yes,NaN,26-100,...,Somewhat easy,No,No,Some of them,Some of them,No,No,Don't know,No,NaN
1255,2015-09-26 01:07:35,32,Male,United States,IL,No,Yes,Yes,Often,26-100,...,Somewhat difficult,No,No,Some of them,Yes,No,No,Yes,No,NaN
1256,2015-11-07 12:36:58,34,male,United States,CA,No,Yes,Yes,Sometimes,More than 1000,...,Somewhat difficult,Yes,Yes,No,No,No,No,No,No,NaN
1257,2015-11-30 21:25:06,46,f,United States,NC,No,No,No,NaN,100-500,...,Don't know,Yes,No,No,No,No,No,No,No,NaN


# Data Cleaning

### How Clean is Our Data?
- The data is imperfect, but still usable after some cleaning. We see in the 'Age' column that there are outlier ages (like 8, 11, -1) which are likely due to the individual's freedom in inputting their age and human error in doing so. Furthermore, there are columns in which the data values are not uniform. For example, the gender column contains values like 'M', 'F', 'Female', 'female', 'mail', etc. In terms of variable relevance, some columns (like Timestamp) are unnecessary for our analysis. 

### Cleaning Our Data 🧼
- To address some of the issues mentioned above, we took the following steps:
    1. We dropped unnecessary variables (columns) from our dataset. We concurred that these variables were not relevant to our analysis:
        - Timestamp
        - State
        - wellness_program
        - seek_help
        - anonymity
        - phys_health_consequence
        - coworkers
        - supervisor
        - mental_health_interview
        - phys_health_interview
        - mental_vs_physical
        - comments
    2. We removed outliers in the 'Age' column. To do so, we took the lower .05% and upper .05% of the 'Age' range, which removed all ages below 18 and above 60. This addressed the issue of the extreme outliers and only took out 1% of our observations.
    3. To address the lack of uniformity of the 'Gender' column, we replaced all values that didn't match 'M' or 'F' exactly with the appropriate labeling of either 'M', 'F', or 'NB' (Non-Binary).
    4. After completing the above, we decided to adapt some of the columns with strictly Yes/No values to follow one-hot encoding. That is, we replaced 'Yes' values with a 1, and 'No' values with a 0 for ease of later calculations and analysis.
    5. We then re-ordered the columns to place columns with string data (non-numerical values) at the beginning, and then put our one-hot encoded columns at the end. This is done for ease of interpretation

### Pre-Processing Steps

- To come to our data-cleaning conclusions, we first looked at the dtypes of each column to ensure that qualitative values were represented as strings or objects and quantitative values were formatted as numerical representations.
- Then, we looked at the values in each column individually. 
    - In doing so, we found the 'un-clean' formatting of the 'Gender' column as well as the outliers in the 'Age' column.
    - We also found that some of the columns containing Yes/No values might serve us better if they were converted to 0/1 values, which is why we performed one-hot encoding


In [4]:
#Dropping Columns that are not necessary for analysis

tech_data = tech_data.drop(columns = ['Timestamp', 'state','wellness_program',\
                                      'seek_help', 'anonymity', 'phys_health_consequence',\
                                      'coworkers', 'supervisor', 'mental_health_interview',\
                                      'phys_health_interview', 'mental_vs_physical', 'comments'])





In [5]:
#Renaming Columns for Ease of Understanding
tech_data.columns = ['Age', 'Gender', 'Country', 'Self Employed',\
                     'Family History', 'Sought Treatment', 'Work Interfere',\
                    'Number Employees', 'Work Remotely', 'Tech Company',\
                    'Offer Benefits', 'Knowledge of Care Options', 'How Easy to take Medical Leave',\
                    'Mental Health Consequences', 'Observed Consequences']








In [6]:
#Remove upper and lower .05% of Ages to filter out outliers
lower = tech_data['Age'].quantile(.005)

upper = tech_data['Age'].quantile(.995)

tech_data = tech_data[(tech_data['Age']>=lower) & (tech_data['Age']<=upper)]

In [7]:
#Taking care of most gender inputs
gender_expression = lambda x: re.sub(r'^Male|^male|^.* Male|^.* male|M |^m$| M', 'M', x)
gender_expression2 = lambda x: re.sub(r'Female|female|^f$|Woman|Female[.]*|cis-female/femme|Female (trans)|Female (cis)|woman|Femake|Cis Female|Trans-female|queer/she/they| F|F ', 'F', x)

tech_data['Gender'] =(tech_data['Gender'].apply(gender_expression)).apply(gender_expression2)

In [8]:
for_replacement = ['M', 'M', 'M', 'M',\
                   'M', 'NB', 'M', 'NB',\
                   'NB', 'NB', 'NB', 'F',\
                   'M', 'NB', 'NB', 'M', \
                   'M', 'M', 'M', 'M',\
                   'M', 'F', 'M', 'M',\
                   'NB', 'F', 'NB', 'F',\
                   'M', 'F', 'F', 'M',\
                   'M', 'F', 'M', 'M',\
                   'M']
    

In [9]:
#takes care of the rest of the gender inputs
counter = 0
for i in tech_data[(tech_data['Gender']!= 'M') & (tech_data['Gender']!='F')]['Gender'].index:
    tech_data['Gender'].loc[i] = for_replacement[counter]
    counter +=1

/var/folders/7g/5dngg9xn7pxb7j2xdsxfbldr0000gp/T/ipykernel_41338/3636781357.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tech_data['Gender'].loc[i] = for_replacement[counter]


In [10]:
#one-hot encoding of strictly yes/no columns for ease of computation
tech_data[['Self Employed', 'Family History', 'Sought Treatment',\
           'Work Remotely', 'Tech Company', 'Observed Consequences']] \
= tech_data[['Self Employed', 'Family History', 'Sought Treatment', \
             'Work Remotely', 'Tech Company', 'Observed Consequences']]\
.replace('Yes', 1).replace('No', 0)




In [11]:
# Ordering columns for ease of interpretation
ordered_columns = ['Age', 'Gender', 'Country', 'Number Employees',\
                   'Work Interfere', 'Offer Benefits', 'Knowledge of Care Options',\
                  'How Easy to take Medical Leave', 'Mental Health Consequences',\
                  'Self Employed', 'Family History', 'Sought Treatment', 'Work Remotely', 'Tech Company',\
                  'Observed Consequences']

tech_data = tech_data[ordered_columns]

In [12]:
tech_data = tech_data.reset_index().drop(columns = 'index')

In [13]:
tech_data = tech_data[tech_data['Tech Company']!=0]

In [14]:
tech_data = tech_data.drop(columns = 'Tech Company')

In [21]:
tech_data['Work Interfere'].unique()

array(['Often', 'Rarely', 'Never', 'Sometimes', nan], dtype=object)

In [24]:
def ord_encoder_interfere(string):
    if string == 'Often':
        return 4
    elif string == 'Sometimes':
        return 3
    elif string=='Rarely':
        return 2
    elif string == 'Never':
        return 1

In [25]:
def ord_encoder_consequence(string):
    if string == 'No':
        return -1
    elif string == 'Maybe':
        return 0
    elif string == 'Yes':
        return 1

In [28]:
tech_data['Work Interfere Ordinal'] = tech_data['Work Interfere'].apply(ord_encoder_interfere)

In [29]:
tech_data['Mental Health Consequences Ordinal'] = tech_data['Mental Health Consequences'].apply(ord_encoder_consequence)

In [30]:
tech_data

,Age,Gender,Country,Number Employees,Work Interfere,Offer Benefits,Knowledge of Care Options,How Easy to take Medical Leave,Mental Health Consequences,Self Employed,Family History,Sought Treatment,Work Remotely,Observed Consequences,Work Interfere Ordinal,Mental Health Consequences Ordinal
0,37,F,United States,6-25,Often,Yes,Not sure,Somewhat easy,No,NaN,0,1,0,0,4.0,-1
2,32,M,Canada,6-25,Rarely,No,No,Somewhat difficult,No,NaN,0,0,0,0,2.0,-1
3,31,M,United Kingdom,26-100,Often,No,Yes,Somewhat difficult,Yes,NaN,1,1,0,1,4.0,1
4,31,M,United States,100-500,Never,Yes,No,Don't know,No,NaN,0,0,1,0,1.0,-1
5,33,M,United States,6-25,Sometimes,Yes,Not sure,Don't know,No,NaN,1,0,0,0,3.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240,29,M,United States,100-500,Sometimes,Yes,Yes,Don't know,Yes,0.0,1,1,1,0,3.0,1
1242,26,M,United Kingdom,26-100,NaN,No,No,Somewhat easy,No,0.0,0,1,0,0,NaN,-1
1243,32,M,United States,26-100,Often,Yes,Yes,Somewhat difficult,No,0.0,1,1,1,0,4.0,-1
1244,34,M,United States,More than 1000,Sometimes,Yes,Yes,Somewhat difficult,Yes,0.0,1,1,0,0,3.0,1
